Это тестовое задание для прохода на стажировку DS от Avito by Прокопьев Дмитрий.

Импорты и первый просмотр данных


In [3]:
!pip install gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.6/26.6 MB 58.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 64.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.2/38.2 MB 14.5 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.16.2
    Uninstalling scipy-1.16.2:
      Successfully uninstalled scipy-1.16.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatib

In [1]:
import pandas as pd
import numpy as np
import math
from transformers import AutoTokenizer, AutoModel
import torch
from gensim.models import KeyedVectors

In [3]:
data = pd.read_csv('data.txt', names=['id', 'text_no_spaces'], usecols=[0, 1], skiprows=1)
data.head(10)

,id,text_no_spaces
0,0,куплюайфон14про
1,1,ищудомвПодмосковье
2,2,сдаюквартирусмебельюитехникой
3,3,новыйдивандоставканедорого
4,4,отдамдаромкошку
5,5,работавМосквеудаленно
6,6,куплютелевизорPhilips
7,7,ищугрузчиковдляпереезда
8,8,ремонтквартирподключ
9,9,куплюноутбукHP


Глобально используем словарь, идём по буковкам, склеиваем в токен считаем скор токена, минимизируем скор

In [5]:
model = KeyedVectors.load_word2vec_format("cc.ru.300.vec") #тут надо скачать этот словарик

In [12]:
def word_score(word, model):
    try:
        vec = model.get_vector(word)
        return np.linalg.norm(vec)
    except KeyError:
        return 0

def segment_with_dp(text, model, max_word_len=20):
    n = len(text)
    dp = [(+float('inf'), [])] * (n + 1)
    dp[0] = (0, [])

    for i in range(1, n + 1):
        curr = +float('inf')
        current_best_split = None
        for j in range(max(0, i - max_word_len), i):
            word = text[j:i]
            score = word_score(word, model)
            if score == 0:
                continue
            cur_score = dp[j][0] + score
            if cur_score < curr:
                curr = cur_score
                current_best_split = dp[j][1] + [word]
        dp[i] = (curr, current_best_split)

    return dp[n][1]



samples = data.head(10)['text_no_spaces']
for s in samples:
    print(s, "->", segment_with_dp(s, model))

куплюайфон14про -> ['куплю', 'айфон', '14', 'про']
ищудомвПодмосковье -> ['ищу', 'дом', 'в', 'Подмосковье']
сдаюквартирусмебельюитехникой -> ['сдаю', 'кварти', 'рус', 'мебелью', 'и', 'техникой']
новыйдивандоставканедорого -> ['новый', 'диван', 'доставка', 'недорого']
отдамдаромкошку -> ['отдам', 'даром', 'кошку']
работавМосквеудаленно -> ['рабо', 'тав', 'Москве', 'удаленно']
куплютелевизорPhilips -> ['куплю', 'телевизор', 'Philips']
ищугрузчиковдляпереезда -> ['ищу', 'грузчиков', 'для', 'переезда']
ремонтквартирподключ -> ['ремонт', 'квартир', 'под', 'ключ']
куплюноутбукHP -> ['куплю', 'ноутбук', 'HP']


посмотрим что получилось

In [13]:
def get_answer(text):
    parts = segment_with_dp(text, model)
    spaces_positions = []
    total_length = 0
    for part in parts[:-1]:
        total_length += len(part)
        spaces_positions.append(total_length)
    return spaces_positions

ans_df = pd.DataFrame()
ans_df['id'] = data['id']
ans_df['predicted_positions'] = data['text_no_spaces'].astype(str).apply(get_answer)
ans_df.to_csv('submission.csv', index=False, header=True)

ans_df.head(10)

,id,predicted_positions
0,0,"[5, 10, 12]"
1,1,"[3, 6, 7]"
2,2,"[4, 10, 13, 20, 21]"
3,3,"[5, 10, 18]"
4,4,"[5, 10]"
5,5,"[4, 7, 13]"
6,6,"[5, 14]"
7,7,"[3, 12, 15]"
8,8,"[6, 13, 16]"
9,9,"[5, 12]"


в итоге получили f1 скор 76%, думаю для такого алгоритма довольно неплохо